## IRIS Task

In [227]:
import numpy as np
import matplotlib.pyplot as plt

In [228]:
setosaTrainingVec = []
versicolorTrainingVec = []
virginicaTrainingVec = []

setosaTestVec = []
versicolorTestVec = []
virginicaTestVec = []

TrainingSize = 30

with open('iris_data/class_1', 'r') as file:
    for line in file:
        if len(setosaTrainingVec) < TrainingSize:
            setosaTrainingVec.append([float(x) for x in line.split(',')])
        else:
            setosaTestVec.append([float(x) for x in line.split(',')])

with open('iris_data/class_2', 'r') as file:
    for line in file:
        if len(versicolorTrainingVec) < TrainingSize:
            versicolorTrainingVec.append([float(x) for x in line.split(',')])
        else:
            versicolorTestVec.append([float(x) for x in line.split(',')])

with open('iris_data/class_3', 'r') as file:
    for line in file:
        if len(virginicaTrainingVec) < TrainingSize:
            virginicaTrainingVec.append([float(x) for x in line.split(',')])
        else:
            virginicaTestVec.append([float(x) for x in line.split(',')])

In [229]:
setosaTrainingVec = np.array(setosaTrainingVec)
versicolorTrainingVec = np.array(versicolorTrainingVec)
virginicaTrainingVec = np.array(virginicaTestVec)

setosaTestVec = np.array(setosaTestVec)
versicolorTestVec = np.array(versicolorTestVec)
virginicaTestVec = np.array(virginicaTestVec)

X = np.concatenate((setosaTrainingVec, versicolorTrainingVec, virginicaTrainingVec), axis=0)
X_test = np.concatenate((setosaTrainingVec, versicolorTrainingVec, virginicaTrainingVec), axis=0)

In [230]:
np.random.seed(42)

x = np.array([-3, -5, 3, 5])
W = np.random.uniform(-1, 1, (3,4))
T = np.concatenate((np.tile([1,0,0], (30,1)), np.tile([0,1,0], (30,1)), np.tile([0,0,1], (30,1))))

w0 = np.array([0, 0, 0])

def g(x, W): # Eq 20 on vector form classification compendium
    zk = np.matmul(W, x) + w0
    gk = 1 / (1 + np.exp(-zk))
    
    return gk

print(g(x, W))

[0.2016787  0.99851996 0.2963353 ]


In [231]:
def MSE(X, T, W): # Eq 19 in classification compendium
    
    mseSum = 0
    for i in range(np.size(X, 0)):
        xk = X[i,:]
        tk = T[i,:]
        gk = g(xk, W)
        
        mseSum += (1/2)* np.linalg.norm((gk - tk), 2)
    
    return mseSum

def gradMSE(X, T, W): # Eq 22
    
    gradMSESum = np.zeros((3,4))
    
    for i in range(np.size(X, 0)):
        xk = X[i,:]
        tk = T[i,:]
        gk = g(xk, W)
        
        a = np.multiply((gk - tk), gk)
        b = np.multiply(a, np.ones(3).T - gk)

        gradMSESum += np.outer(b, xk)

    return gradMSESum

print(MSE(X, T, W))
print(gradMSE(X, T, W))
    

45.18014179191745
[[ 7.95901545e+00  3.42083100e+00  6.20898718e+00  2.06421884e+00]
 [-3.08533830e-02 -1.40728168e-02 -2.18044515e-02 -6.78543257e-03]
 [ 1.72922038e+01  1.24244658e+01  1.84568659e+00 -1.25142789e+00]]


### Linesearch

In [232]:
tolerance = 1e-4
alpha = 0.3

np.random.seed(42)
W = np.random.uniform(-1, 1, (3,4))

while np.linalg.norm(gradMSE(X, T, W), ord='fro') > tolerance:
    print(f"Norm: {np.linalg.norm(gradMSE(X, T, W), ord='fro')}, MSE: {MSE(X, T, W)}")
    W = W - alpha * gradMSE(X, T, W)
    
print(np.linalg.norm(gradMSE(X, T, W), ord='fro'))
print(MSE(X, T ,W))


Norm: 24.00463491628757, MSE: 45.18014179191745
Norm: 0.04482919026988716, MSE: 39.99698520930121
Norm: 0.0495989759837512, MSE: 39.99666787657253
Norm: 0.05547536597741215, MSE: 39.99627725760188
Norm: 0.06288426983083141, MSE: 39.99578522768344
Norm: 0.0724971492177998, MSE: 39.99514748003768
Norm: 0.08543352146267219, MSE: 39.99429019982849
Norm: 0.10369816430444338, MSE: 39.9930813078987
Norm: 0.13123640984662915, MSE: 39.99126107895144
Norm: 0.1768949502852875, MSE: 39.98824750923937
Norm: 0.26472776841009754, MSE: 39.98245850502508
Norm: 0.4843934255788056, MSE: 39.96798867591133
Norm: 1.4583963320587727, MSE: 39.90336650602318
Norm: 19.344598243517964, MSE: 37.79840088723714
4.227098854480517e-05
35.355336505102066
